In [36]:
from langchain.chat_models import ChatOpenAI
from llama_index import LLMPredictor, ServiceContext, PromptHelper
from langchain import PromptTemplate
from langchain.prompts import ChatPromptTemplate

import openai,os,requests,json,bs4
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [37]:
url = "https://park.ajinomoto.co.jp/recipe/card/705645/"

In [38]:
template_string = """\
## Recipe Format
```json
{recipe_format}
```

## Instruction
[Recipe Format]に従ってレシピデータを作成してください。

"""

In [39]:
recipe_format = {
    "name": "レシピ名",
    "description": "レシピの説明",
    "cooking_time": "int: minutes",
    "appliances": [
        "str: 調理器具"
    ],
    "serves": "int: 人前",
    "nutrition": {
        "calorie": "int: カロリー(kcal)", 
        "protein": "float: たんぱく質(gram)", 
        "lipid": "float: 脂質(gram)",
        "carbohydrate": "float: 炭水化物(gram)",
        "sugar": "float: 糖質(gram)",
        "dietary_fiber": "float: 食物繊維(gram)",
        "salt": "float: 塩分(gram)"
    },
    "ingredients": [
        {
            "name": "str: 材料名",
            "amount": "str: 分量表記",
            "g": "int: 分量表記から推定した重量(gram)"
        }
    ],
    "steps": [
        "str: 手順の説明"
    ]
}

In [40]:
prompt_template = ChatPromptTemplate.from_template(template_string)
print(prompt_template.messages[0].prompt.input_variables)

['recipe_format']


In [41]:
max_input_size = 4096
num_output = 2048  #2048に拡大
max_chunk_overlap = 0.2
prompt_helper = PromptHelper(
    max_input_size,
    num_output,
    max_chunk_overlap
)
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, max_tokens=2048))
service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor, 
    prompt_helper=prompt_helper
)

In [42]:
from llama_index import GPTVectorStoreIndex
from llama_index.readers import BeautifulSoupWebReader

documents = BeautifulSoupWebReader().load_data(urls=[url])
index = GPTVectorStoreIndex.from_documents(documents,  service_context=service_context) 
query_engine = index.as_query_engine()

In [43]:
answer = query_engine.query(prompt_template.format(recipe_format=json.dumps(recipe_format,indent=2,ensure_ascii=False)))
print(answer)

Based on the new context, here is an example of the recipe format:

```json
{
  "name": "さやいんげんの煮物",
  "description": "さやいんげんを煮た和風の煮物です。",
  "cooking_time": 20,
  "appliances": [
    "鍋"
  ],
  "serves": 2,
  "nutrition": {
    "calorie": 150,
    "protein": 5,
    "lipid": 7,
    "carbohydrate": 15,
    "sugar": 5,
    "dietary_fiber": 5,
    "salt": 2
  },
  "ingredients": [
    {
      "name": "水",
      "amount": "2カップ",
      "g": 480
    },
    {
      "name": "酒",
      "amount": "大さじ2",
      "g": null
    },
    {
      "name": "ほんだし",
      "amount": "小さじ1",
      "g": null
    },
    {
      "name": "砂糖",
      "amount": "大さじ1",
      "g": 20
    },
    {
      "name": "みりん",
      "amount": "大さじ1",
      "g": null
    },
    {
      "name": "しょうゆ",
      "amount": "大さじ2",
      "g": null
    },
    {
      "name": "さやいんげん（ゆで）",
      "amount": "適量",
      "g": 200
    },
    {
      "name": "AJINOMOTO サラダ油",
      "amount": "適量",
      "g": null
    }
  ],
  "steps": [
    